In [1]:
import pandas as pd
from catboost import CatBoostRegressor
from catboost import EShapCalcType, EFeaturesSelectionAlgorithm

import numpy as np

from sklearn.model_selection import cross_validate
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression, Lasso

from src.utils import prediction_on_folds, train_nocv, compute_metrics_by_method
from src.outputs import save_fold_predictions, save_nofolds_predictions
from src.manipulations import categorical_features
from src.labels_paper import labels

import os
from src.consts import DATADIRECTORY

# Read data

### Read data for implicit substrates

In [2]:
df = pd.read_csv(os.path.join(DATADIRECTORY,'implicit_dataset.csv'))
target = pd.read_csv(os.path.join(DATADIRECTORY,'implicit_PT.csv'))

### Read data for explicit substrates

In [3]:
df_es = pd.read_csv(os.path.join(DATADIRECTORY,'explicit_dataset.csv'))
target_es = pd.read_csv(os.path.join(DATADIRECTORY,'explicit_PT.csv'))

In [4]:
# prepare data for LinearRegression and Lasso
# remove possible NaNs and perform one-hot encodings
df_nonans = df.dropna(axis=1)
df_es_nonans = df_es.dropna(axis=1)
no_nans_common_columns = list(set(df_nonans.columns).intersection(df_es_nonans.columns))
df_nonans = df_nonans[no_nans_common_columns]
df_es_nonans = df_es_nonans[no_nans_common_columns]

cat_features = categorical_features(df)
cat_features_nonans = categorical_features(df_nonans)
df_nonans_onehot = pd.get_dummies(df_nonans, columns=cat_features_nonans)
df_es_nonans_onehot = pd.get_dummies(df_es_nonans, columns=cat_features_nonans)

# Training: Cross Validation using all features

In [5]:
scoring = {
    'mean_squared_error': 'neg_mean_squared_error',
    'mean_absolute_error': 'neg_mean_absolute_error',
    'r2': 'r2',
}

cb_hyperparams = {
    'max_depth': 5, 
    'n_estimators': 2000,
    'eta':0.05
}

cb = CatBoostRegressor(verbose=False, cat_features=cat_features, 
                       **cb_hyperparams)
cv_cb = cross_validate(cb, df, target, cv=5, scoring=scoring, 
                           return_train_score=True, 
                           return_estimator=True,
                           return_indices=True)

lasso = Lasso(alpha=0.2)
cv_lasso = cross_validate(lasso, df_nonans_onehot, target, cv=5, scoring=scoring,                            
                              return_train_score=True, 
                              return_estimator=True,
                              return_indices=True)
lr = LinearRegression(fit_intercept=True)
cv_lr = cross_validate(lr, df_nonans_onehot, target, cv=5, scoring=scoring,                            
                              return_train_score=True, 
                              return_estimator=True,
                              return_indices=True)

In [6]:
# Train on all data without CV for better prediction for Explicit Substrates
nocv_cb = train_nocv(cb, df, target)
nocv_lasso = train_nocv(lasso, df_nonans_onehot, target)
nocv_lr = train_nocv(lr, df_nonans_onehot, target)

In [7]:
# Make predictions on whole dataset
csv_path_all_features = save_fold_predictions(cv_objs=[cv_lr, cv_lasso, cv_cb],
                                            labels=['MLR', 'LASSO', 'ML'],
                                            dfs=[df_nonans_onehot, df_nonans_onehot, df],
                                            targets=[target, target, target],
                                            filename=f'predictions_fit_all_features_implicit_PTastarget',
                                            substrate='implicit')

In [8]:
# Metrics for implicit substrates. PT used as target.
compute_metrics_by_method(csv_path_all_features, substrate='implicit', filename='Metrics_all_features_implicit_PTastarget.txt')

implicit MLR	MAErr=15.9361	RMSErr=92.6781	$R^2$	=-1.69
implicit LASSO	MAErr=13.9672	RMSErr=27.4073	$R^2$	=0.76
implicit ML	MAErr=7.8437	RMSErr=12.1313	$R^2$	=0.95


In [9]:
# Metrics for explicit substrates
csv_path_all_features_es = save_nofolds_predictions(estimators=[nocv_lr, nocv_lasso, nocv_cb],
                                        labels=['MLR', 'LASSO', 'ML'],
                                        dfs=[df_es_nonans_onehot, df_es_nonans_onehot, df_es],
                                        targets=[target_es, target_es, target_es],
                                        filename=f'predictions_fit_all_features_explicit_PTastarget',
                                        substrate='explicit')

In [10]:
compute_metrics_by_method(csv_path_all_features_es, substrate='explicit', filename='Metrics_all_features_explicit_PTastarget.txt')

explicit MLR	MAErr=3378.2675	RMSErr=3388.9552	$R^2$	=-3836.90
explicit LASSO	MAErr=888.1310	RMSErr=896.5401	$R^2$	=-267.60
explicit ML	MAErr=24.3965	RMSErr=28.5992	$R^2$	=0.73


# Feature Selection

In [11]:
# Feature selection with CV
num_features_to_selects = [2, 3, 5, 6, 7, 8, 9, 10, 12, 15, 17, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 80, 90, 100]

mse_feature = []
mae_feature = []
r2_feature = []
features_selection = {}

for num_features_to_select in num_features_to_selects:
    mod = CatBoostRegressor(cat_features=cat_features)
    summary = mod.select_features(df, target,
                          features_for_select=list(range(df.shape[1])),
                          num_features_to_select=num_features_to_select,
                          steps=20,
                          algorithm=EFeaturesSelectionAlgorithm.RecursiveByShapValues,
                          shap_calc_type=EShapCalcType.Regular,
                          train_final_model=True,
                          plot=False, logging_level='Silent')

    features_selection[num_features_to_select] = summary['selected_features_names']
    df_selected = df[summary['selected_features_names']]
    cb_mod = CatBoostRegressor(**cb_hyperparams, cat_features=cat_features, verbose=False)
    cv_cb_fetures = cross_validate(cb_mod, df_selected, target, cv=5, scoring=scoring, 
                           return_train_score=True, 
                           return_estimator=True,
                           return_indices=True)
    y_pred_folds, y_test_folds = prediction_on_folds(cv_cb_fetures, df_selected, target)
    mae = mean_absolute_error(y_test_folds, y_pred_folds)
    mse = mean_squared_error(y_test_folds, y_pred_folds)
    r2 = r2_score(y_test_folds, y_pred_folds)
    mse_feature.append(mse)
    mae_feature.append(mae)
    r2_feature.append(r2)
    print('****')
    print(f'REQUIRED NUMBER OF FEATURES {num_features_to_select}')
    print(summary['selected_features_names'])
    print(f'MAE={mae} MSE={mse} RMSE={np.sqrt(mse)} R2={r2}')
    print('****')

****
REQUIRED NUMBER OF FEATURES 2
['sr_A1_dz2-r2DOWN', 'bader_A2']
MAE=14.805762343975104 MSE=501.77555572024875 RMSE=22.40034722320725 R2=0.8429216961110191
****
****
REQUIRED NUMBER OF FEATURES 3
['sr_A1_dz2-r2DOWN', 'bader_A2', 'sr_A1_dosmodeldyzDOWN_E_above']
MAE=11.133701341126287 MSE=256.5201706764474 RMSE=16.016247084646494 R2=0.919697655926402
****
****
REQUIRED NUMBER OF FEATURES 5
['sr_A1_dz2-r2DOWN', 'sr_A1_dosmodeldz2-r2DOWN_integral_below_1.0', 'sr_A2_dosmodeldz2-r2DOWN_integral_below_1.0', 'bader_A2', 'sr_A1_dosmodeldyzDOWN_E_above']
MAE=9.721514507325706 MSE=209.92293788096643 RMSE=14.488717606502187 R2=0.9342846843497532
****
****
REQUIRED NUMBER OF FEATURES 6
['sr_A1_dz2-r2DOWN', 'sr_A1_dosmodeldz2-r2DOWN_integral_below_1.0', 'sr_A2_dosmodeldz2-r2DOWN_integral_below_1.0', 'bader_A2', 'sr_A1_dosmodeldyzDOWN_E_above', 'sr_A2_dosmodeldxyUP_E_below']
MAE=9.45692646329389 MSE=202.73715931176778 RMSE=14.238579961209888 R2=0.9365341560446255
****
****
REQUIRED NUMBER OF FEAT

# Build models for list selected 25 features

In [12]:
N_FEATURES = 25
FEATURES = features_selection[N_FEATURES]
print(f'FEATURES: {FEATURES}')
df_selected = df[FEATURES].copy()

df_es_selected = df_es[FEATURES]
# prepare data for LinearRegression
# remove NaNs
# do one-hot encodings
df_nonans_selected = df_selected.dropna(axis=1)
df_es_nonans_selected = df_es_selected.dropna(axis=1)
no_nans_common_columns = list(set(df_nonans_selected.columns).intersection(df_es_nonans_selected.columns))
df_nonans_selected = df_nonans_selected[no_nans_common_columns]
df_es_nonans_selected = df_es_nonans_selected[no_nans_common_columns]

cat_features_selected = categorical_features(df_selected)
cat_features_nonans_selected = categorical_features(df_nonans_selected)
df_nonans_onehot_selected = pd.get_dummies(df_nonans_selected, columns=cat_features_nonans_selected)
df_es_nonans_onehot_selected = pd.get_dummies(df_es_nonans_selected, columns=cat_features_nonans_selected)

cat_features_nonans_selected = categorical_features(df_nonans_selected)
df_nonans_selected_onehot = pd.get_dummies(df_nonans_selected, columns=cat_features_nonans_selected)

FEATURES: ['BG_ind_down', 'dimer_len', 'sr_A1_dz2-r2DOWN', 'sr_A2_dz2-r2DOWN', 'sr_A2_dxzUP', 'sr_A1_dosmodeldyzUP_integral_below_1.0', 'sr_A1_dosmodeldz2-r2UP_integral_below_1.0', 'sr_A1_dosmodeldyzDOWN_integral_below_1.0', 'sr_A1_dosmodeldz2-r2DOWN_integral_below_1.0', 'sr_A1_dosmodeldxzDOWN_integral_below_1.0', 'sr_A2_dosmodeldyzDOWN_integral_below_1.0', 'sr_A2_dosmodeldz2-r2DOWN_integral_below_1.0', 'sr_A2_dosmodeldxzDOWN_integral_below_1.0', 'sr_A2_dosmodeldz2-r2DOWN_integral_above_1.0', 'bader_A2', 'sr_A1_dosmodeldyzUP_E_below', 'sr_A1_dosmodeldyzUP_E_above', 'sr_A1_dosmodeldyzUP_peak_above', 'sr_A1_dosmodeldz2-r2UP_E_above', 'sr_A1_dosmodeldyzDOWN_E_above', 'sr_A2_dosmodeldxyUP_E_below', 'sr_A2_dosmodeldx2-y2UP_E_below', 'sr_A2_dosmodeldyzDOWN_E_above', 'sr_A2_dosmodeldz2-r2DOWN_peak_above', 'sr_A2_dosmodeldxzDOWN_E_above']


In [13]:
# print selected features in LaTeX format
[labels(f) for f in FEATURES]

['$BG_\\text{ind}^{\\downarrow}$',
 '$d_{len}$',
 '$DOS_{B, d_{z^2}}^{\\downarrow}$',
 '$DOS_{T, d_{z^2}}^{\\downarrow}$',
 '$DOS_{T, d_{xz}}^{\\uparrow}$',
 '$I_{B, d_{yz}}^{\\uparrow -}$',
 '$I_{B, d_{z^2}}^{\\uparrow -}$',
 '$I_{B, d_{yz}}^{\\downarrow -}$',
 '$I_{B, d_{z^2}}^{\\downarrow -}$',
 '$I_{B, d_{xz}}^{\\downarrow -}$',
 '$I_{T, d_{yz}}^{\\downarrow -}$',
 '$I_{T, d_{z^2}}^{\\downarrow -}$',
 '$I_{T, d_{xz}}^{\\downarrow -}$',
 '$I_{T, d_{z^2}}^{\\downarrow +}$',
 '$q_T$',
 '$E_{B, d_{yz}}^{\\uparrow -}$',
 '$E_{B, d_{yz}}^{\\uparrow +}$',
 '$D_{B, d_{yz}}^{\\uparrow +}$',
 '$E_{B, d_{z^2}}^{\\uparrow +}$',
 '$E_{B, d_{yz}}^{\\downarrow +}$',
 '$E_{T, d_{xy}}^{\\uparrow -}$',
 '$E_{T, d_{x^2-y^2}}^{\\uparrow -}$',
 '$E_{T, d_{yz}}^{\\downarrow +}$',
 '$D_{T, d_{z^2}}^{\\downarrow +}$',
 '$E_{T, d_{xz}}^{\\downarrow +}$']

In [14]:
lasso = Lasso(alpha=0.1)
cv_lasso_selected = cross_validate(lasso, df_nonans_selected_onehot, target, cv=5, scoring=scoring,                            
                              return_train_score=True, 
                              return_estimator=True,
                              return_indices=True)

cv_lr_selected = cross_validate(lr,df_nonans_selected_onehot, target, cv=5, scoring=scoring,                            
                              return_train_score=True, 
                              return_estimator=True,
                              return_indices=True)

cb = CatBoostRegressor(verbose=False, cat_features=cat_features_selected)
cv_cb_selected = cross_validate(cb, df_selected, target, cv=5, scoring=scoring, 
                           return_train_score=True, 
                           return_estimator=True,
                           return_indices=True)

In [15]:
nocv_cb_selected = train_nocv(cb, df_selected, target)
nocv_lasso_selected = train_nocv(lasso, df_nonans_selected_onehot, target)
nocv_lr_selected = train_nocv(lr, df_nonans_selected_onehot, target)

In [16]:
csv_path_selected = save_fold_predictions(cv_objs=[cv_lr_selected, cv_lasso_selected, cv_cb_selected],
                                        labels=['MLR', 'LASSO', 'ML'],
                                        dfs=[df_nonans_selected_onehot, df_nonans_selected_onehot, df_selected],
                                        targets=[target, target, target],
                                        filename=f'predictions_fit_selected_features_implicit_PTastarget',
                                        substrate='implicit')

## Trainig metrics when predicting PT: Reproduce Figure S7

In [17]:
compute_metrics_by_method(csv_path_selected, substrate='implicit', filename='Metrics_selected_features_implicit_PTastarget.txt')

implicit MLR	MAErr=15.2278	RMSErr=24.4894	$R^2$	=0.81
implicit LASSO	MAErr=15.3229	RMSErr=25.3436	$R^2$	=0.80
implicit ML	MAErr=7.8078	RMSErr=12.2224	$R^2$	=0.95


**Conclusion**: Machine Learning model (ML), MLR and LASSO have larger performance when PT is used as target value. 

Metrics above corresponds to Figure S7 and is moreover discussed in the Manuscript in lines 421-430. Small differences in RMSErr between Supplementary Information and this code is due to stochastic effects (different train/val/test splits etc). 

In [18]:
csv_path_selected_es = save_nofolds_predictions(estimators=[nocv_lr_selected, nocv_lasso_selected, nocv_cb_selected],
                                        labels=['MLR', 'LASSO', 'ML'],
                                        dfs=[df_es_nonans_onehot_selected, df_es_nonans_onehot_selected, df_es_selected],
                                        targets=[target_es, target_es, target_es],
                                        filename=f'predictions_fit_selected_features_explicit_PTastarget',
                                        substrate='explicit')        

In [19]:
compute_metrics_by_method(csv_path_selected_es, substrate='explicit', filename='Metrics_selected_features_explicit_PTastarget.txt')

explicit MLR	MAErr=1115.8158	RMSErr=1123.5927	$R^2$	=-420.87
explicit LASSO	MAErr=1077.1710	RMSErr=1085.0627	$R^2$	=-392.43
explicit ML	MAErr=28.9476	RMSErr=32.0898	$R^2$	=0.66


Table above shows the prediction for explicit substrates. When compared with Table 1 one can see that R2 coefficient for ML is larger then when MAE is used as target.